# How to configure your Spark Application

#### spark-submit --class <CLASS_NAME> --num-executors ? --executor-cores ? --executor-memory ? ....
![alt text](./img/spark-1.png)

In [2]:
# Importing Classes to launch a Spark Context
from pyspark import SparkContext, SparkConf

In [7]:
# Creating a configuration object
conf = SparkConf()

### What are some commonly used configuration properties?

* Spark Master
* Spark Application name
* spark.driver.memory
* spark.executor.memory
* spark.executor.cores
* spark.executor.instances
* spark.driver.maxResultSize

In [8]:
conf.setMaster('local[*]') # set to YARN if running on a cluster
conf.setAppName('My Test Application')
conf.set('spark.driver.memory', '4g')
conf.set('spark.executor.memory', '2g')
conf.set('spark.executor.cores', '7')
conf.set('spark.executor.instances', '2')
conf.set('spark.driver.maxResultSize', '12g')

In [9]:
sc = SparkContext(conf=conf)

<hr>
### What is Driver Memory?

<p>When we start our spark application with spark submit command,
a driver will start and that driver will contact spark master
to launch executors and run the tasks.
Basically, Driver is a representative of our application and does all the communication with Spark.</p>

### What is Executor Memory?

<p> The amount of memory to be allocated to an executor, spark adds an addition overhead
to the amount of memory which is 7% of requested memory before allocating the space </p>
#### spark.executor.memory = requested_memory + max(384MB, 0.07*requested_memory)
<hr>

## Setting up a cluster

<p> Let's assume our cluster configuration is as follows:<br>
Total Nodes: <b>10</b> <br>
Total Cores per Node: <b>16</b><br>
Total Memory per Node: <b>64GB</b><br>
</p>

#### Fat Executors

In [14]:
# Configuration with fat executor
conf = SparkConf()
conf.set('spark.driver.memory', '48gb')
conf.set('spark.executor.memory','58gb') # Not 64gb, because we need some memory for executor overhead
conf.set('spark.executor.cores', '15') # Not 16 cores, leaving one core for daemons to run smoothly
conf.set('spark.executor.instances', '9') # One executor per node, except on master.

<p> This configuration launches 9 instances of executors, <br>
each taking up one node in the cluster </p><br>
<p> Lauching a Spark Job with this configuration is a bad idea,<br>
as most application read and write data from a distributed file system<br>
like HDFS, S3 or GFS. Such a configuration will lead to a lot of <br>
garbage collection and reduced parallelism </p>
<hr>

#### Lean Executors

In [19]:
# Configuration with lean executor
conf = SparkConf()
conf.set('spark.driver.memory', '48gb')
conf.set('spark.executor.memory','3gb') # why not 4gb?
conf.set('spark.executor.cores', '1') 
conf.set('spark.executor.instances', '135') # why not 144?

<p> In this configuration, there is an executor running on each core<br>
In total we have 10*16 cores = 160, but 10 of them are on driver.<br>
So, 150 cores, can we launch 150 executors??</p>

<p> Out of the 64gb memory on each node, so if 16 executors are running,<br>
can we can give each executor 64/16 = 4gb memory? </p>
<br>

<p> With a lean configuration containing many executors, <br>
we'd not be able to take advantage of runnning multiple tasks within<br>
the same JVM and cached/broadcasted variables will be replicated.<br>
</p>
<hr>

#### What is the best configuration?

<p> The best configuration depends a lot on the type of<br>
spark application you are running.</p>
<br>
<p> However, its a good idea to have atleast 5 cores per executor,<br>
and choose executor memory accordingly.</p><br>

In [21]:
conf = SparkConf()
conf.set('spark.driver.memory', '48gb')
conf.set('spark.executor.memory','10gb') 
conf.set('spark.executor.cores', '5') 
conf.set('spark.executor.instances', '30')

![alt text](./img/aws-emr.png)

### Dynamic Allocation in Spark

#### spark.dynamicAllocation.enabled
<p>It allows spark to spawn and kill executors dynamically based on the application,<br>
However, you still need to provide some initial condition as well as bounds for<br>
number of executors.<br>
</p>

In [32]:
from pyspark import SparkConf
conf = SparkConf()
conf.set('spark.dynamicAllocation.enabled', 'true')
conf.set("spark.dynamicAllocation.minExecutors", "2");
conf.set("spark.dynamicAllocation.maxExecutors", "10");
conf.set("spark.dynamicAllocation.initialExecutors", "10");

In [33]:
sc = SparkContext(conf=conf)

### Running Spark using Zeppelin

<p>If you are running Spark on a web-based notebook service like Zeppelin,<br>
SparkContext is already initialised, and to change the configuration<br>
you have to use sc._conf object.<p>

![alt text](./img/zeppelin.png)

In [34]:
sc._conf.getAll()

[('spark.app.id', 'local-1592161710152'),
 ('spark.executor.memory', '2g'),
 ('spark.driver.memory', '4g'),
 ('spark.app.name', 'My Test Application'),
 ('spark.dynamicAllocation.maxExecutors', '10'),
 ('spark.executor.id', 'driver'),
 ('spark.dynamicAllocation.minExecutors', '2'),
 ('spark.driver.maxResultSize', '12g'),
 ('spark.driver.port', '50680'),
 ('spark.driver.host', '192.168.0.101'),
 ('spark.executor.cores', '7'),
 ('spark.rdd.compress', 'True'),
 ('spark.executor.instances', '2'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.dynamicAllocation.initialExecutors', '10'),
 ('spark.submit.deployMode', 'client'),
 ('spark.dynamicAllocation.enabled', 'true'),
 ('spark.ui.showConsoleProgress', 'true')]

#### Starting a Spark SQL Session

In [35]:
spark = SparkSession(sparkContext=sc)